# Preamble 😴

Parallel computing is great. Let's start with a simple example.


*Note*: I use pretty bad importing practices throughout this notebook. Usually, everything should be imported at the top, but I make my imports within sections for visual clarity!


I'll start with a wrapper for [tqdm](https://github.com/tqdm/tqdm), my favorite progress bar. If you don't have it installed, it'll just return an empty iterator.

In [1]:
try:
    from tqdm import tqdm
except ImportError:
    print(
        "tqdm is not installed on this system, \
            so progress bars will be unavailable. \
            If you would like the progress bar feature, \
            please see the tqdm installation docs: \
            https://github.com/tqdm/tqdm#installation"
    )
    # tqdm isn't available, so we define a "do-nothing" class.
    class tqdm(object):
        """
        Dummy class to replace the tqdm progress bar.
        """

        def __init__(self, **kwargs):
            """
            Initializes the dummy class. Any kwargs that are
            passed to the "real" tqdm don't matter if tqdm
            isn't installed.
            """
            pass

        def update(self, val):
            """
            Dummy update method.
            """
            pass

# A function 🔧

In [2]:
def function(long_list, verbose=True):
    """
    A function that performs a very bad sort of a very long list.
    
    I think it performs as like O(n^2), but I'm not good at algorithms!
    
    Inputs
    ------
        :long_list: (list) a list to be sorted.
        :verbose: (bool) print that we're starting or no?
        
    Outputs
    --------
        :long_list_sorted: (list) the input list, now sorted.
    """
    long_list_sorted = []
    
    long_list_sorted += [long_list[0]]
    
    if verbose:
        print('Starting sort!')
    
    for i, value in enumerate(long_list[1:]):
#         pdb.set_trace()
        
        for j, compare_value in enumerate(long_list_sorted):

            if value <= compare_value:
                long_list_sorted.insert(j, value)
                break
    
            #at the end, still need to add the value!
            elif j == len(long_list_sorted) - 1:
                long_list_sorted += [value]
                break
                
            else:
                continue
            
                    
    return long_list_sorted
                
                
    

In [217]:
function([5, 2, 3, 3, 4])

Starting sort!


[2, 3, 3, 4, 5]

In [218]:
%%timeit

function([1,2,3,3,5,1,4,2], verbose=False)

7.68 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Oop! It's slow! 😳

In [219]:
from tqdm import tqdm

In [220]:
arr = [1, 8532, 1, 86321, 3, 95]

long_arr = []

for i in range(1000):
    long_arr += arr

In [221]:
nested_arr = []

for i in range(7):
    nested_arr += [long_arr]

In [222]:
%%time

for i in range(7):
    function(long_arr, verbose=False)

CPU times: user 10.9 s, sys: 75 ms, total: 10.9 s
Wall time: 12.1 s


# Parallel programming time 😎

In [135]:
from multiprocessing import Pool
from functools import partial

In [166]:
%%time

num_processes = 2

with Pool(num_processes) as p:
    func = partial(function, verbose=False)

    result = list(p.imap(func, nested_arr))

CPU times: user 15.3 ms, sys: 16.3 ms, total: 31.6 ms
Wall time: 6.92 s


In [167]:
%%time

num_processes = 3

with Pool(num_processes) as p:
    func = partial(function, verbose=False)

    result = list(p.imap(func, nested_arr))

CPU times: user 15.3 ms, sys: 17.8 ms, total: 33.1 ms
Wall time: 6.73 s


# Multithreading 🤹

In [3]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
url = 'https://en.wikipedia.org/wiki/List_of_tautological_place_names'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

In [4]:
from urllib.error import HTTPError
from queue import Queue
from threading import Thread


In [7]:
hrefs = []

for a in soup.find_all('a', href=True):
    href = a['href']
    if href[:5] == '/wiki':
        href = 'https://en.wikipedia.org/' + href
        
    if href[:3] == '/w/' or href[0] == '#':
        continue
        
    if href[:2] == '//':
        href = href[2:]
    
    hrefs += [href]

In [13]:
from tqdm import tqdm

In [14]:
%%time

for href in tqdm(hrefs):
    try:
        html = urlopen(href) 
        soup = BeautifulSoup(html, 'html.parser')
    except:
        pass

100%|██████████| 745/745 [08:05<00:00,  1.53it/s]

CPU times: user 3min 55s, sys: 5.79 s, total: 4min
Wall time: 8min 6s


In [5]:
class DownloadWorker(Thread):

    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue

    def run(self):
        while True:
            # Get the work from the queue and expand the tuple
            href = self.queue.get()
            try:
                html = urlopen(href) 
                soup = BeautifulSoup(html, 'html.parser')
            except:
                pass
            finally:
                self.queue.task_done()

In [16]:
%%time

queue = Queue()


# Create the worker threads
for x in range(2):
    worker = DownloadWorker(queue)
    # Setting daemon to True will let the main thread exit even though the workers are blocking
    worker.daemon = True
    worker.start()
    
# Put the tasks into the queue as a tuple
for href in hrefs:
    queue.put((href))
# Causes the main thread to wait for the queue to finish processing all the tasks
queue.join()

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


CPU times: user 3min 45s, sys: 6.5 s, total: 3min 52s
Wall time: 5min 13s


# Bonus: Numba 🔢

In [172]:
from numba import njit
import numpy as np

In [207]:
def function(long_list, verbose=True):
    """
    A function that performs a very bad sort of a very long list.
    
    I think it performs as like O(n^2), but I'm not good at algorithms!
    
    Inputs
    ------
        :long_list: (list) a list to be sorted.
        :verbose: (bool) print that we're starting or no?
        
    Outputs
    --------
        :long_list_sorted: (list) the input list, now sorted.
    """
#     long_list = np.array(long_list)
    long_list = long_list * long_list
    
    for value in [13,14,15]:
        long_list = value * long_list
            
                    
    return long_list
                

In [213]:
@njit
def function_jitted(long_list, verbose=True):
    """
    A function that performs a very bad sort of a very long list.
    
    I think it performs as like O(n^2), but I'm not good at algorithms!
    
    Inputs
    ------
        :long_list: (list) a list to be sorted.
        :verbose: (bool) print that we're starting or no?
        
    Outputs
    --------
        :long_list_sorted: (list) the input list, now sorted.
    """
#     long_list = np.array(long_list)
    long_list = long_list * long_list
    
    for value in [13,14,16]:
        long_list = value * long_list
                
    return long_list

In [209]:
%%timeit

for i in range(7):
    function(np.array([1,2,3]), verbose=False)

38.3 µs ± 4.53 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [214]:
%%timeit

for i in range(7):
    function_jitted(np.array([1,2,3]), verbose=False)

27.4 µs ± 6.96 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [215]:
%%timeit

for i in range(7):
    function_jitted(np.array([1,2,3]), verbose=False)

25.3 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Bonus: OpenMP

We won't actually go through OpenMP — but we can take a look at some of the syntax to get a feel for it.

(Code borrowed from [here](https://gribblelab.org/teaching/CBootCamp/A2_Parallel_Programming_in_C.html))

In [ ]:
#include <omp.h>

main ()  {

int var1, var2, var3;

Serial code 
      .
      .
      .

Beginning of parallel section. Fork a team of threads.
Specify variable scoping 

#pragma omp parallel private(var1, var2) shared(var3)
  {

  Parallel section executed by all threads 
        .
        .
        .

  All threads join master thread and disband 

  }  

Resume serial code 
      .
      .
      .

}